Esta pauta cuenta con una forma de modelar el problema, existen múltiples maneras distintas de abordarlo y no necesariamente una es mejor que otra, lo que importa es que apliquen la lógica y buenas prácticas de programación.

En la pauta hay uso de sintaxis un poco más avanzada a lo que se espera para efectos de este curso, el objetivo es que la pauta también signifique un aprendizaje para ustedes, no quiere decir que se espere exactamente lo mismo que en la pauta. Ante cualquier duda sobre la pauta pueden ocupar las issues del curso.

# Consideraciones
 - los vehiculos aparecen con una distribucion poisson en cada sentido, lambda de autos por minuto
 - la simulacion va a ser en un estrecho de 100.000 metros y se va a medir en minutos
 - las pistas 1 son las lentas y las 2 son las rapidas
 - todo auto que adelanta vuelve apenas puede a la pista lenta
 - un auto no puede pasar por sobre otro no estar en la misma posicion


In [ ]:
import random
import math
from typing import List

In [ ]:
class Automovil:
    def __init__(self, pos: int):
        self.id = None
        self.prob_adelanto: int = None
        self.posicion: int = pos

        #todas en km/h
        self.velocidad_actual: int = None
        self.velocidad_adelanto: int = None
        self.velocidad_normal: int = None

    def adelantar(self) -> bool:
        return random.randint(1, 100) < self.prob_adelanto

    def avance(self, maximo:int, fin:int) -> bool:
                                                            # km/h -> mt/min
        self.posicion = int(min(maximo-1, self.posicion + (self.velocidad_actual*16.6)))
        return self.posicion >= fin

    def set_vel_adelanto(self):
        self.velocidad_actual = self.velocidad_adelanto
    
    def set_vel_normal(self):
        self.velocidad_actual = self.velocidad_normal
        
class Liviano(Automovil):
    cls_id = 0
    def __init__(self, pos: int):
        super().__init__(pos)
        self.id = Liviano.cls_id
        Liviano.cls_id += 1
        self.prob_adelanto: int = 55
        self.velocidad_adelanto: int = random.randint(111,125)
        self.velocidad_normal: int = random.randint(90,110)
        self.velocidad_actual = self.velocidad_normal

    def nombre(self):
        return f'L{self.id}'

class Pesado(Automovil):
    cls_id = 0
    def __init__(self, pos: int):
        super().__init__(pos)
        self.id = Pesado.cls_id
        Pesado.cls_id += 1
        self.prob_adelanto: int = 0
        self.velocidad_adelanto: int = None
        self.velocidad_normal: int = random.randint(90,110)
        self.velocidad_actual = self.velocidad_normal

    def nombre(self):
        return f'P{self.id}'

    def set_vel_adelanto(self):
        self.set_vel_normal()

class Carrera(Automovil):
    cls_id = 0
    def __init__(self, pos: int):
        super().__init__(pos)
        self.id = Carrera.cls_id
        Carrera.cls_id += 1
        self.prob_adelanto: int = 90
        self.velocidad_adelanto: int = random.randint(120,150)
        self.velocidad_normal: int = random.randint(100,119)
        self.velocidad_actual = self.velocidad_normal

    def nombre(self):
        return f'C{self.id}'

    def set_vel_normal(self):
        self.set_vel_adelanto()

In [ ]:
def obtener_posicion(vehiculo):
    return vehiculo.posicion

In [ ]:
class Sentido:
    def __init__(self, d: str, t: float, s: int):
        # [499, 498, 497, ..., 3, 2, 1, 0] la posicion relativa entre los vehiculos es cual 
        # esta mas adelantado que el resto (los mas adelantados a la izq)
        self.lento: List[Automovil] = []
        self.rapido: List[Automovil] = []
        self.sig: int = 0.0
        self.direccion = d
        self.tasa = t
        self.tamano_simular = s

    def nuevo_vehiculo(self, pos: int):
        # prob_l prob_p prob_c
        probs = [0.45,0.35,0.2]
        seleccion = random.random()

        if seleccion <= sum(probs[:1]):
            return Liviano(pos)
        elif seleccion <= sum(probs[:2]):
            return Pesado(pos)
        elif seleccion <= sum(probs[:3]):
            return Carrera(pos)

    def sgte(self):
        self.sig += (-1*math.log(1-random.random())/self.tasa)

    def crear(self):
        v = 0

        if self.sig <= 1:
            while self.sig < 1:
                v += 1
                self.sgte()
        self.sig -= 1
        
        pos = 0
        for _ in range(v):
            self.lento.append(self.nuevo_vehiculo(pos))
            # le puse -2 para que el auto no crea que tiene que adelantar
            pos -= 2
        if self.sig <= 0:
            self.sgte()

        print(f'Se materializaron {v} autos desde el {self.direccion}')
    
    def ordenar(self): # idealmente tendria que ir agregandolas en orden y no tener que ordenar a cada rato
        self.lento = sorted(self.lento, key=obtener_posicion, reverse=True)
        self.rapido = sorted(self.rapido, key=lambda x: x.posicion, reverse=True)

    def sub_adelantar(self, lentos: List[Automovil], rapidos: List[Automovil]):
        posiciones_rapidos = set([x.posicion for x in rapidos])
        cambio = []

        for i in range(1, len(lentos)):
            if lentos[i].posicion == lentos[i-1].posicion-1 and lentos[i].adelantar():
                if lentos[i].posicion not in posiciones_rapidos:
                    cambio.append(i)

        for i in cambio[::-1]: # voy de atras para adelante para no romper los indices de la lista
            rapidos.append(lentos.pop(i))
            rapidos[-1].set_vel_adelanto()

        return lentos, rapidos

    def adelantar(self):
        self.ordenar()
        self.lento, self.rapido = self.sub_adelantar(self.lento, self.rapido)

    def sub_avanzar(self, autos: List[Automovil]):
        eliminar = []
        for i, v in enumerate(autos):
            if v.avance(self.tamano_simular+1 if i == 0 else autos[i-1].posicion, self.tamano_simular):
                eliminar.append(i)
        for i in eliminar[::-1]:
            del autos[i]
        return autos

    def avanzar(self):
        self.ordenar()
        self.lento = self.sub_avanzar(self.lento)
        self.rapido = self.sub_avanzar(self.rapido)

    def sub_volver(self, lentos: List[Automovil], rapidos: List[Automovil]):
        #obtengo todas las posiciones de los autos en la pista lenta
        posiciones_lento = set([x.posicion for x in lentos])
        cambio = []

        for i, v in enumerate(rapidos):
            if v.posicion not in posiciones_lento:
                cambio.append(i)

        for i in cambio[::-1]: # voy de atras para adelante para no romper los indices de la lista
            lentos.append(rapidos.pop(i))
            lentos[-1].set_vel_normal()

        return lentos, rapidos

    def volver(self):
        self.ordenar()
        self.lento, self.rapido = self.sub_volver(self.lento, self.rapido)

In [ ]:
class Simulador:
    def __init__(self, norte: int, sur: int):
        # tasa en autos por minuto
        self.tasa_norte: float = norte
        self.tasa_sur: float = sur
        #en metros
        self.tamano_simular = 100000 

        self.norte = Sentido('Norte', self.tasa_norte, self.tamano_simular)
        self.sur = Sentido('Sur', self.tasa_sur, self.tamano_simular)

    def pprint(self):
        print(f'N1-{[f"{x.nombre()}#{x.posicion}" for x in self.norte.lento]}')
        if len(self.norte.rapido) + len(self.sur.rapido) > 0:
            print(f'N2-{[f"{x.nombre()}#{x.posicion}" for x in self.norte.rapido]}')
            print(f'S2-{[f"{x.nombre()}#{x.posicion}" for x in self.sur.rapido][::-1]}')
        print(f'S1-{[f"{x.nombre()}#{x.posicion}" for x in self.sur.lento][::-1]}')
        print()

    def step(self):
        self.norte.crear()
        self.sur.crear()
        
        self.norte.adelantar()
        self.sur.adelantar()

        self.norte.ordenar()
        self.sur.ordenar()
        self.pprint()
        
        self.norte.avanzar()
        self.sur.avanzar()

        self.norte.volver()
        self.sur.volver()

    def simulate(self, horas: int = 2, minutos: int = 5):
        steps: int = horas*60 + minutos
        self.norte.sgte()
        self.sur.sgte()

        for i in range(10):
            print(f'{round((i/steps)*100, 1)}% Minuto {i}')
            self.step()

In [ ]:
a = Simulador(5, 3)
a.simulate()

0.0% Minuto 0
Se materializaron 11 autos desde el Norte
Se materializaron 2 autos desde el Sur
N1-['P0#0', 'L0#-2', 'L1#-4', 'P1#-6', 'L2#-8', 'P2#-10', 'C0#-12', 'L3#-14', 'L4#-16', 'P3#-18', 'C1#-20']
S1-['L5#-2', 'P4#0']

0.8% Minuto 1
Se materializaron 4 autos desde el Norte
Se materializaron 3 autos desde el Sur
N1-['P0#1593', 'L1#1591', 'P1#1590', 'L2#1535', 'P2#1534', 'L4#1527', 'P3#1526', 'P5#0', 'P6#-2', 'L6#-4', 'C2#-6']
N2-['L0#1592', 'C0#1533', 'L3#1532', 'C1#1525']
S2-[]
S1-['C3#-4', 'L8#-2', 'L7#0', 'L5#1658', 'P4#1743']

1.6% Minuto 2
Se materializaron 10 autos desde el Norte
Se materializaron 6 autos desde el Sur
N1-['L0#3534', 'L3#3507', 'P0#3186', 'L1#3185', 'P1#3184', 'L2#3078', 'P2#3077', 'L4#3070', 'P3#3069', 'P5#1726', 'P6#1725', 'L6#1724', 'L9#0', 'C4#-2', 'L10#-4', 'L11#-6', 'P7#-8', 'P8#-10', 'C5#-12', 'L12#-14', 'P9#-16', 'L13#-18']
N2-['C0#3533', 'C1#3506', 'C2#1723']
S2-['C3#1706']
S1-['P13#-10', 'P12#-8', 'P11#-6', 'P10#-4', 'C7#-2', 'C6#0', 'L8#1707', 'L7#